## Description, Imports, and Configuration

In [ ]:
#!/usr/bin/env python
# coding: utf-8

"""
FunBurd: Functional Burden Test for Gene Set Association Analysis
Author: Sayeh Kazem
Python Version: 3.10.2

Rationale:
FunBurd (Functional Burden Test) is a statistical method designed to estimate the association between
Copy Number Variants (CNVs) and complex traits. It works by aggregating the effect of rare CNVs
within functionally related gene sets to increase statistical power and identify key biological pathways
underlying complex phenotypes.

Script Description:
This script performs a statistical analysis to quantify the association between CNVs in predefined
gene sets and a continuous or binary phenotype. The analysis pipeline for each gene set involves
the following steps: 

1.  **CNV Count Generation:** For each individual, a count is calculated for CNV events
    overlapping with genes both "inside" and "outside" the specified gene set.
2.  **Regression Model Fitting:** A regression model is fitted to assess the effect of the "inside" and "outside"
    CNV counts on the phenotype. The script dynamically selects between Generalized Least Squares (GLS) for
    continuous traits and Firth logistic regression for binary traits to mitigate potential separation issues.
3.  **Parallelized Execution:** The analysis is parallelized across multiple gene sets using the `multiprocessing`
    library, optimizing performance in high-performance computing (HPC) environments.

The final output is a structured, tab-separated file containing the effect size, standard deviation, and p-value for
each gene set's "inside" and "outside" effect.

"""

# --- 1. Library Imports ---
import os
import sys
import time
import numpy as np
import pandas as pd
from multiprocessing import Pool
import statsmodels.formula.api as sm
from patsy import dmatrices
from firthlogist import FirthLogisticRegression

# --- 2. Configuration Parameters ---
# PHENOTYPE_FILE = "Phenotypic_info_Toy/phenotype1_toy.tsv"
# PHENOTYPE_COLUMN = "phenotype1"
# PHENOTYPE_NAME = "phenotype1"

PHENOTYPE_FILE = "Phenotypic_info_Toy/phenotype3_toy.tsv"
PHENOTYPE_COLUMN = "phenotype3"
PHENOTYPE_NAME = "phenotype3"


GENE_SET_DIRECTORY = "Gene_Sets/GeneSet_HPA_tissue_fantom_TDEP_nTPM"
GENE_SET_NAME = "GeneSet_HPA_tissue_fantom"
COVARIATES = ' + age + Sex + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10'

# File paths for input and output data.
CURRENT_DIR = os.getcwd()

PATH_GENE_FILES = os.path.join(CURRENT_DIR, GENE_SET_DIRECTORY)
PATH_GENE_INFO = os.path.join(CURRENT_DIR,"Genotypic_info_Toy", "cnv_gene_df_toy.tsv")
PATH_CNV_INDIVIDUAL = os.path.join(CURRENT_DIR,"Genotypic_info_Toy", "cnv_individual_df_toy.tsv")
PATH_CLEAN_DATA = os.path.join(CURRENT_DIR, "Phenotypic_info_Toy","clean_individual_df_toy.tsv")

OUTPUT_PATH = os.path.join(CURRENT_DIR,'Outcome_Toy', f"Effect_sizes_{GENE_SET_NAME}_{PHENOTYPE_NAME}.tsv")

# Number of CPU cores for parallel processing.
CPU_COUNT = int(os.environ.get("SLURM_CPUS_PER_TASK", 6))

print("Notebook setup complete. Proceed to the next cell to load data.")

Notebook setup complete. Proceed to the next cell to load data.


## Data Loading and Preprocessing

In [2]:
print("Loading all required data files...")

cnv_individual_df = pd.read_csv(PATH_CNV_INDIVIDUAL, sep='\t')
cnv_individual_df['individual'] = cnv_individual_df['individual'].str[:7].astype(str)

phenotype_individual_df = pd.read_csv(PHENOTYPE_FILE, sep='\t')
phenotype_individual_df['individual'] = phenotype_individual_df['individual'].astype(str)

cnv_gene_df = pd.read_csv(
    PATH_GENE_INFO,
    sep='\t', usecols=['CHR', 'START', 'STOP', 'TYPE', 'proportion_gene_overlap', 'gene_id']
)

clean_individual_df = pd.read_csv(PATH_CLEAN_DATA, usecols=["individual", "Ancestry"],sep='\t')
clean_individual_df['individual'] = clean_individual_df['individual'].str[:7].astype(str)

# Filter for individuals with 'EUR' ancestry and merge with phenotype data
phenotype_individual_df = pd.merge(phenotype_individual_df, clean_individual_df[clean_individual_df['Ancestry'] == 'EUR'], on='individual', how='inner')

print("Data loading complete.")


Loading all required data files...
Data loading complete.


In [3]:
phenotype_individual_df.head(5)

individual  age  Sex      PC1       PC2       PC3       PC4       PC5  \
0       ID_1   49    0  62.4282   4.70889 -2.764410 -0.211590 -3.377320   
1       ID_3   61    0 -10.4589   4.60139 -4.373810  0.411185 -8.152480   
2       ID_4   65    1 -13.9528   3.45020 -2.415750  1.946210 -4.913340   
3       ID_5   59    0 -10.8248 -98.60360 -0.215284 -0.867102 -0.319268   
4       ID_6   44    0 -15.4783       NaN -0.188110 -0.746203  1.298200   

       PC6       PC7       PC8       PC9      PC10  phenotype3 Ancestry  
0  9.36288  2.095320 -0.089103 -3.138240 -1.682450         0.0      EUR  
1  7.56120  2.053210 -3.552020  1.804400 -0.403587         0.0      EUR  
2      NaN  3.613120 -0.359041  3.331650  3.355560         0.0      EUR  
3  1.06374 -0.336161 -0.894880  3.267970 -2.974150         0.0      EUR  
4 -1.25487 -5.670860 -0.402316 -0.043084 -0.755178         1.0      EUR

In [4]:
phenotype_individual_df.shape

(803, 15)

In [5]:
cnv_gene_df.head(5)

gene_id    CHR      START       STOP TYPE  proportion_gene_overlap
0  ENSG00000187555  chr20   49574017   49912726  DEL                 0.166811
1  ENSG00000166105   chr1  169740986  170528913  DUP                 1.000000
2  ENSG00000176256   chr1  169740986  169856661  DEL                 0.950124
3  ENSG00000243137   chr1  169548053  170665664  DUP                 1.000000
4  ENSG00000049130   chr1  169740986  170528913  DUP                 1.000000

In [6]:
cnv_gene_df.shape

(313455, 6)

In [7]:
cnv_individual_df.head(5)

individual    CHR      START       STOP TYPE
0      ID_38  chr10   68456145   68550490  DEL
1     ID_474  chr11   91481728   91669648  DEL
2     ID_523  chr18   38514376   38681926  DEL
3     ID_597  chr18    7067652    7575300  DUP
4     ID_947   chr5  147968858  148319491  DUP

In [8]:
cnv_individual_df.shape

(396, 5)

In [9]:
clean_individual_df.head(5)

individual Ancestry
0     ID_104      EUR
1     ID_517      EUR
2     ID_617      EUR
3     ID_737      EUR
4     ID_936      EUR

In [10]:
clean_individual_df.shape

(850, 2)

## Functions

In [11]:
# ==================================================================================================
## Data Processing and Aggregation
# ==================================================================================================

def annotate_gene_set_status(data, gene_column_name, gene_list):
    """
    Annotates genes as being either 'inside' or 'outside' a specified gene list.

    Args:
        data (pd.DataFrame): DataFrame containing gene information.
        gene_column_name (str): The name of the column containing gene identifiers.
        gene_list (list): A list of gene identifiers to check against.

    Returns:
        pd.DataFrame: The original DataFrame with two new columns, 'inside' and 'outside',
                      indicating gene set membership (1 for true, 0 for false).
    """
    data["inside"] = data[gene_column_name].isin(gene_list).astype(int)
    data["outside"] = (~data[gene_column_name].isin(gene_list)).astype(int)
    return data

def aggregate_cnv_counts(cnv_individual_df, cnv_gene_df, phenotype_individual_df):
    """
    Counts CNV events for each individual based on genes inside and outside the set.

    Args:
        cnv_individual_df (pd.DataFrame): DataFrame with individual-level CNV data.
        cnv_gene_df (pd.DataFrame): DataFrame with gene-level CNV data and annotations.
        phenotype_individual_df (pd.DataFrame): DataFrame with individual phenotype data.

    Returns:
        pd.DataFrame: A DataFrame with CNV counts (inside/outside for DEL/DUP) aggregated
                      by individual, merged with phenotype data.
    """
    cnv_gene_del_df = cnv_gene_df[cnv_gene_df["TYPE"] == "DEL"]
    cnv_gene_dup_df = cnv_gene_df[cnv_gene_df["TYPE"] == "DUP"]
    
    cnv_individual_del_df = cnv_individual_df[cnv_individual_df["TYPE"] == "DEL"]
    cnv_individual_dup_df = cnv_individual_df[cnv_individual_df["TYPE"] == "DUP"]

    def process_cnv_type(cnv_subset, gene_subset):
        """Helper function to aggregate counts for a single CNV type (DEL or DUP)."""
        gene_subset = gene_subset.copy()
        gene_subset['position'] = gene_subset.apply(lambda row: f"{row['CHR']}--{row['START']}--{row['STOP']}", axis=1)
        gene_subset.set_index('position', inplace=True)
        gene_counts = gene_subset[['inside', 'outside']].groupby(level="position").sum()

        cnv_subset = cnv_subset.copy()
        cnv_subset['position'] = cnv_subset.apply(lambda row: f"{row['CHR']}--{row['START']}--{row['STOP']}", axis=1)
        cnv_positions = cnv_subset[['position', 'individual']]

        merged_counts = pd.merge(cnv_positions, gene_counts, on="position", how="outer")
        individual_counts = merged_counts[['inside', 'outside']]
        individual_counts.index = merged_counts["individual"]
        return individual_counts.groupby(level="individual").sum()

    count_individual_del_cnvs = process_cnv_type(cnv_individual_del_df, cnv_gene_del_df)
    count_individual_dup_cnvs = process_cnv_type(cnv_individual_dup_df, cnv_gene_dup_df)

    final_counts = pd.merge(count_individual_dup_cnvs, count_individual_del_cnvs, on="individual", how="outer", suffixes=('_DUP', '_DEL'))
    
    final_counts.columns = [
        "inside_DUP", "outside_DUP",
        "inside_DEL", "outside_DEL"
    ]

    final_counts = pd.merge(phenotype_individual_df, final_counts, on="individual", how="inner")
    final_counts.fillna(0, inplace=True)
    
    return final_counts


# ==================================================================================================
## Statistical Modeling (Firth Logistic Regression for Binary traits and Linear regression for Continues Traits)
# ==================================================================================================

class FirthLogisticRegressionResults:
    """A class to hold Firth Logistic Regression results for consistent output formatting."""
    def __init__(self, coefficients_series, std_errors_series, p_values_series):
        self.params = coefficients_series
        self.bse = std_errors_series
        self.pvalues = p_values_series

def fit_firth_logistic_regression(formula, data):
    """
    Performs Firth logistic regression with robust data handling and result formatting.

    Args:
        formula (str): A patsy-compatible formula string for the model.
        data (pd.DataFrame): The DataFrame containing the model variables.

    Returns:
        FirthLogisticRegressionResults: An object containing the model's coefficients,
                                        standard errors, and p-values.
    """
    y, X0 = dmatrices(formula, data, return_type='dataframe')
    X = X0.drop('Intercept', axis=1)
    feature_names = X.columns.tolist()
    
    y = y.values.ravel()
    X = X.values
    
    # Firth logistic regression model
    fl0 = FirthLogisticRegression()
    fl = fl0.fit(X, y)
    
    variable_names =  ['Intercept'] + feature_names 
    variable_def =  feature_names +['Intercept']
    
    # Extract coefficients and intercept
    intercept = fl.intercept_
    coefficients = fl.coef_
    coefficients_series = pd.Series(np.concatenate(( [intercept],coefficients)), index=variable_names) 
    # Extract std
    std_errors = fl.bse_
    std_errors_series = pd.Series(std_errors, index=variable_def)
    # Extract pvals
    p_values = fl.pvals_
    p_values_series = pd.Series(p_values, index=variable_def)
    # Reorder the std and pvalues
    std_errors_series = std_errors_series.reindex(variable_names)
    p_values_series = p_values_series.reindex(variable_names)
    
    return FirthLogisticRegressionResults(coefficients_series,std_errors_series,p_values_series)


def fit_model_and_extract_results(data, cnv_type, pheno_col, covariates):
    """
    Fits the appropriate regression model and returns a DataFrame of key results.

    Args:
        data (pd.DataFrame): The DataFrame containing all data for the model.
        cnv_type (str): The CNV type ("DEL" or "DUP").
        pheno_col (str): The name of the phenotype column.
        covariates (str): A string of additional covariates for the formula.

    Returns:
        pd.DataFrame: A DataFrame with the model's effect size, standard error,
                      and p-value. Returns an empty DataFrame on error.
    """
    formula = f"{pheno_col} ~ inside_{cnv_type} + outside_{cnv_type}{covariates}"
    
    try:
        # if pheno_col in ['Final_phenotype_value_info_GP', 'Final_phenotype_info_Depression', 'Final_phenotype_value_Hypertention', 'Final_phenotype_value_MoodSwings', 'Final_phenotype_value_loneliness', 'Final_phenotype_value_RiskTaking', 'Final_phenotype_value_Miserableness', 'Final_phenotype_value_Irritability', 'Final_phenotype_value_GuiltyFeelings', 'Final_phenotype_value_SubjectiveWellBeing']:
        #     regression_results = fit_firth_logistic_regression(formula, data)
        # else:
        #     regression_results = sm.gls(formula, data=data).fit()
        is_binary_trait = np.array_equal(data[pheno_col].dropna().unique(), [0, 1])
        if is_binary_trait:
            # Run Firth logistic regression
            regression_results = fit_firth_logistic_regression(formula, data)
        else:
    # Run GLM for continuous traits
            regression_results = sm.gls(formula, data=data).fit()
        
    except Exception as e:
        print(f"Error running model for {cnv_type}: {e}")
        return pd.DataFrame()

    results_df = pd.DataFrame({
        'Effectsize': regression_results.params.values,
        'se': regression_results.bse.values,
        'pvalue': regression_results.pvalues.values
    })
    
    return results_df

# ==================================================================================================
## Main Analysis Workflow
# ==================================================================================================

def analyze_gene_set_data(list_name):
    """
    Primary worker function that orchestrates the analysis for a single gene list.

    It reads the gene list, processes the data, aggregates CNV counts, and fits
    the appropriate regression models for both DEL and DUP CNV types.

    Args:
        list_name (str): The name of the gene list file to be processed.

    Returns:
        pd.DataFrame: A summary DataFrame of the final regression results for the
                      processed gene list.
    """
    print(f"Processing gene list: {list_name}")
    start_time = time.time()
    
    try:
        gene_list = pd.read_csv(f"{PATH_GENE_FILES}/{list_name}", sep='\t')
        gene_list.columns = ["gene_id"]
        
        gene_info_subset = cnv_gene_df[cnv_gene_df["proportion_gene_overlap"] >= 1].copy()
        gene_info_with_indicators = annotate_gene_set_status(
            gene_info_subset, "gene_id", gene_list.iloc[:, 0].tolist()
        )
        
        combined_counts_df = aggregate_cnv_counts(
            cnv_individual_df,
            gene_info_with_indicators,
            phenotype_individual_df
        )
        
        cleaned_data = combined_counts_df.dropna(subset=[PHENOTYPE_COLUMN])
        
        final_results = []
        
        for cnv_type in ["DEL", "DUP"]:
            results_df = fit_model_and_extract_results(
                data=cleaned_data,
                cnv_type=cnv_type,
                pheno_col=PHENOTYPE_COLUMN,
                covariates=COVARIATES
            )
            
            # Extracts the row corresponding to the 'inside' effect, as per original logic.
            results_df = results_df.iloc[1:2].copy()
            results_df['TYPE'] = cnv_type
            results_df['gene_list_name'] = list_name
            final_results.append(results_df)

        final_list_summary = pd.concat(final_results, ignore_index=True)
        print(f"Finished processing {list_name} in {(time.time() - start_time):.2f} seconds.")
        return final_list_summary
    
    except Exception as e:
        print(f"An error occurred while processing {list_name}: {e}")
        return pd.DataFrame()

## Execution and Output

In [12]:
if __name__ == '__main__':
    
    print("Beginning the gene set association analysis...")

    list_of_gene_files = [f for f in os.listdir(PATH_GENE_FILES) if f.endswith(".tsv")]
    
    start_all = time.time()
    
    with Pool(CPU_COUNT) as pool:
        results = pool.map(analyze_gene_set_data, list_of_gene_files)
    
    final_results_df = pd.concat(results, ignore_index=True)
    
    if not final_results_df.empty:
        final_results_df.to_csv(OUTPUT_PATH, sep="\t", index=False)
        print(f"Analysis successfully completed. Results saved to: {OUTPUT_PATH}")
    else:
        print("No results were generated. The process may have failed.")
        
    print(f"Total execution time: {(time.time() - start_all):.2f} seconds.")

Beginning the gene set association analysis...
Processing gene list: colon.tsvProcessing gene list: corpus_callosum.tsvProcessing gene list: globus_pallidus.tsvProcessing gene list: olfactory_bulb.tsv
Processing gene list: occipital_cortex.tsv



Processing gene list: esophagus.tsv
Finished processing globus_pallidus.tsv in 77.86 seconds.
Processing gene list: small_intestine.tsv
Finished processing colon.tsv in 83.69 seconds.
Processing gene list: thyroid_gland.tsv
Finished processing occipital_cortex.tsv in 86.04 seconds.
Processing gene list: spinal_cord.tsv
Finished processing olfactory_bulb.tsv in 87.51 seconds.
Processing gene list: amygdala.tsv
Finished processing corpus_callosum.tsv in 90.93 seconds.
Processing gene list: occipital_pole.tsv
Finished processing esophagus.tsv in 92.44 seconds.
Processing gene list: hippocampus.tsv
Finished processing thyroid_gland.tsv in 75.41 seconds.
Processing gene list: locus_coeruleus.tsv
Finished processing small_intestine.tsv in 89.04 seco